In [14]:
from tensorflow.keras.models import model_from_json
import glob
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import cv2

In [15]:
# モデルの読み込み
model = model_from_json(open('and.json', 'r').read())

# 重みの読み込み
model.load_weights('and_weight.hdf5')

In [16]:
image_size = 200

In [17]:
def cropping(img, x, y, w, h):
    #(x, y),(x+px, y),(x,y+px),(x+px, y+px)を四隅としてcrop
    cropped = img[y:y+h,x:x+w]
    #サイズを拡大　
    cropped = cv2.resize(cropped, (image_size, image_size))
    return cropped

In [26]:
def face_search(image, name):
    #顔
    face_cascade_path = "haarcascade_frontalface_alt.xml"
    # カスケード分類器を作成
    face_cascade = cv2.CascadeClassifier(face_cascade_path) 

    # 画像を読み込む
    img = cv2.imread(image) 
    # グレースケール化
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 


    # 出力結果用にコピー & BGR化
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    #顔を検知
    faces = face_cascade.detectMultiScale(img_gray)
    
    color = [(255,0,0), (255, 255, 255)]
    
    if len(faces) > 0:
        for (x,y,w,h) in faces:
            X = cropping(img, x, y, w, h)
            X = X.astype('float32')
            X = X / 255.0
            X = X[None, ...]
            prd = model.predict(X).argmax(axis=1)
            
            cv2.rectangle(img,(x,y),(x+w,y+h),color[int(prd)],10)
    else :
        print("認識失敗", image)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    cv2.imwrite("./images/test/ans_" + name + ".JPG", img)

In [27]:
files = glob.glob("./images/test/*.JPG")
for i, file in enumerate(files):
    face_search(file, str(i))